# ニューストピック分類 Base Modelの評価 with CPU



## Google ドライブの接続  
以下を実行。表示されるurlからGoogleアカウントにサイン。表示される認証用のコードを以下に表示されるテキストボックスにコピペ。  
→ サインインしたGoogleアカウントのGoogleドライブがマウントされる。

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


## 主な設定

In [ ]:
data_path = '/content/drive/My Drive/Colab Notebooks/data'   # 作業用フォルダ

## ライブラリのインストール

In [ ]:
!pip install transformers==4.10
!pip install datasets==1.11
!pip install fugashi==1.1
!pip install ipadic==1.0

     |████████████████████████████████| 2.8 MB 7.3 MB/s 
     |████████████████████████████████| 3.3 MB 38.0 MB/s 
     |████████████████████████████████| 636 kB 54.7 MB/s 
     |████████████████████████████████| 895 kB 34.2 MB/s 
     |████████████████████████████████| 50 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 8.6 MB/s 
     |████████████████████████████████| 243 kB 46.2 MB/s 
     |████████████████████████████████| 119 kB 57.2 MB/s 
     |████████████████████████████████| 486 kB 7.8 MB/s 
     |████████████████████████████████| 13.4 MB 6.5 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=f4ef7a1aadb363ea57607c8ba91add48d64967224ff71154490841a370b4fafa
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully bui

In [ ]:
import os
import torch
import pandas as pd
from transformers import BertForSequenceClassification, BertJapaneseTokenizer

## モデルの読み込み


In [ ]:
loaded_model = BertForSequenceClassification.from_pretrained(data_path) 
# loaded_model.cuda()
loaded_tokenizer = BertJapaneseTokenizer.from_pretrained(data_path)

## テストデータの読み込み

In [ ]:
cat_info = pd.read_csv(os.path.join(data_path, "cat_info.csv"), index_col=False)
cat_names = cat_info['name']
cat_names

0    dokujo-tsushin
1    livedoor-homme
2     kaden-channel
3              smax
4        topic-news
5            peachy
6       movie-enter
7      it-life-hack
8      sports-watch
Name: name, dtype: object

In [ ]:
news_test = pd.read_csv(
    os.path.join(data_path, "news_test.csv" ), index_col=False)
news_test.head()

,text,label
0,2008年、”鉄の女”のニックネームを持つ元英国首相マーガレット・サッチャーが、実は認知症に...,5
1,箱根駅伝に出場し、4区を走った東京農業大学・内藤寛人選手の部屋の様子が日本テレビ系情報番組「...,4
2,キヤノンが女子に大人気の小物ブランド「Samantha Thavasa Petit Choi...,5
3,アレクサンドル・デュマの名作文学『三銃士』が、現代の壮大なエンタテインメント作品として生まれ...,5
4,悪質なコンピュータウイルスが氾濫しているWindowsに対して「Macは安全」とされる。Ma...,7


## サンプルデータの予測

In [ ]:
# 形態素解析
MAX_LENGTH = 512
def tokenize(text):
    words = loaded_tokenizer.tokenize(text)
    word_ids = loaded_tokenizer.convert_tokens_to_ids(words)  # インデックスに変換
    word_tensor = torch.tensor([word_ids[:MAX_LENGTH]])  # テンソルに変換
    return word_tensor

In [ ]:
index = 30
sample_text  = news_test.loc[index, 'text']
sample_label = news_test.loc[index, 'label']
print('text :', sample_text)
print('true label:', sample_label, cat_names[sample_label])

x = tokenize(sample_text)
# x = x.cuda()  # GPU対応
y = loaded_model(x)  # 予測
pred = y[0].argmax(-1).item()  # 最大値のインデックス
print("pred label:", pred, cat_names[pred])

text : 「ゴルフ婚活」や「料理婚活」など、ただ食事やお酒を共にするだけでは無く、イベント性をプラスした"婚活"はもはや定番。共通の話題が生まれやすく、自然と仲良くなれるので、女性からもアプローチがしやすい所が魅力的です。そんな中、結婚情報サイト「youbride（ユーブライド）」が3日、代々木公園の清掃を通して交流を深める、地球にやさしいエコと婚活のイベント「クリーンアップ婚活イベント」を開催しました。エコと婚活の融合なんて、新鮮でイマドキ！清々しい秋晴れの日、集まった「youbride」会員は男性20名、女性20名の計40名。「清掃活動って疲れそう…」と思う方が多いかもしれませんが、「クリーンアップ婚活イベント」はあくまで、外の気持ちの良い空気を楽しみながら、街をキレイにしていく"プチ"エコ活動。高すぎるヒールでなければ、ブーツやパンプスでお洒落をして参加できるのが魅力的です。代々木公園の入口から、それぞれチームを作ってイベントがスタート。男性がゴミをつかむトングを、女性がゴミ袋を持つので自然と連携がとれ、2人は急接近。タバコの吸いがらが道に落ちている事が多かった事から「タバコは吸いますか？」「吸いますが、携帯灰皿を持ち歩いています」等と、普通の婚活パーティでは出ない話題も飛び出し、お互いの事をよく知る事が出来ました。エコ活動終了後は、ワインやお料理が楽しめるカフェバーに移動してパーティがスタート。「お疲れ様でした！」の乾杯の後、自由に会話を楽しみました。温泉旅行やギフト券が当たるゲームは、エコ活動で気が合ったパートナー、その時には話せなかった参加者の垣根を越えて大盛り上がり。終始和やかで楽しいパーティに。気の合った者同士はアドレス交換するなど、しっかり婚活の主旨も忘れていませんでした。イベントに参加した方々からは終了後、参加者に感想を聞いたアンケートでは、ほとんどの方が「こういったイベント型の婚活パーティははじめてだったので楽しかったです」「皆さん、自然に関心がある方で色々とお話が出来ました」と好感触。椅子に座ってテーブル越しにお話をするだけでは分からない、人柄を知ることが出来る新鮮な体験だった様です。普通の婚活じゃつまらない、仲良くなれないと嘆く婚活女子の皆さんは今回の「クリーンアップ婚活」の様な“進化型婚活”をチェックしてみてはいかが？・youbrid

# 精度評価と性能評価

In [ ]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm
test_texts  = news_test['text']
test_labels = news_test['label']

In [ ]:
%%time
preds = []
for text in tqdm(test_texts):
    x = tokenize(text)
    # x = x.cuda()  # GPU対応
    y = loaded_model(x)  # 予測
    pred = y[0].argmax(-1).item()  # 最大値のインデックス
    preds.append(pred)

100%|██████████| 1842/1842 [49:49<00:00,  1.62s/it]

CPU times: user 49min 21s, sys: 40.6 s, total: 50min 1s
Wall time: 49min 49s


In [ ]:
print(accuracy_score(test_labels, preds))

0.9381107491856677
